In [42]:
%load_ext autoreload
%autoreload 2

import os
import zarr
import dask.array as da
import napari
import sys
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append('..')
from tracks_interactions.db.db_model import Base, CellDB, TrackDB
import tracks_interactions.graph.family_graph as fg
#from tracks_interactions.widget.event_handler import EventHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
os.environ['NAPARI_PERFMON'] = '0'

In [44]:
labels_zarr_path = r'D:\kasia\tracking\E6_exp\code\tests\example_track_labels.zarr'
labels = da.from_zarr(labels_zarr_path)
labels.shape

(241, 8396, 8401)

In [45]:
#labels = np.array(labels).astype('int32')

In [46]:
# display the labels
viewer = napari.Viewer()
labels_layer = viewer.add_labels(labels,name='Labels')

In [47]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\cells_database_2tables - Copy.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

### Family graph display

In [41]:
from tracks_interactions.graph.family_graph import FamilyGraphWidget
plot_widget = FamilyGraphWidget(viewer,session)
viewer.window.add_dock_widget(plot_widget,area='bottom')

### Navigation widget

In [33]:
from tracks_interactions.widget.track_navigation_widget import TrackNavigationWidget
track_navigation_widget = TrackNavigationWidget(viewer,session)
viewer.window.add_dock_widget(track_navigation_widget,area='right',name='Track Navigation')

### Modification widget

In [34]:
from tracks_interactions.widget.track_modification_widget import TrackModificationWidget
track_modification_widget = TrackModificationWidget(viewer,session)
viewer.window.add_dock_widget(track_modification_widget,area='right',name='Track Modifications')

In [ ]:
def change_position(step_event):

    global viewer

    offset = 10

    #get camera position
    camera_position = viewer.camera.center

    # move by 2 pixels
    viewer.camera.center = (0, camera_position[1] + offset, camera_position[2] + offset)

In [ ]:
viewer.dims.events.current_step.connect(change_position)

In [10]:
tr = session.query(TrackDB).filter_by(track_id=75020).first()
print(tr.t_begin,tr.t_end,tr.parent_track_id,tr.root)

0 17 -1 1603
